In [5]:
import numpy as np
import cv2
from pathlib import Path
import tifffile as tiff
from matplotlib import pyplot as plt
from scipy.ndimage import rotate
import math

In [6]:
pr_num=12

file_amplitude_path_UT = Path(fr'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Panel PEGASO\probetas\1\{pr_num}\{pr_num}_lente_amplitude.tif')
volume_UT= tiff.imread(file_amplitude_path_UT)

In [7]:
def nube_puntos_UT():
    #saca nube puntos UT
    pass

def nube_puntos_XCT():
    #saca nube puntos XCT
    pass


def UT_plane(volume_UT, threshold=60, return_points=False, its_UT = True):
    """
    Fits a plane to the detected surface in an ultrasound volume.

    Parameters:
    - volume_UT: numpy.ndarray, 3D volume with shape (z, y, x)
    - threshold: int, minimum amplitude value to consider a valid point
    - return_points: bool, if True also returns the detected surface points

    Returns:
    - a, b, c: coefficients of the plane z = a*x + b*y + c
    - (optional) surface_coords_np: array of coordinates (z, y, x) meeting the threshold
    """

    y_max = volume_UT.shape[1]  # y-axis (i)
    x_max = volume_UT.shape[2]  # x-axis (j)

    surface_coords = []

    for x in range(x_max):       # j
        for y in range(y_max):   # i
            A_scan = volume_UT[:, y, x]  # signal along z-axis

            max_val = np.max(A_scan)
            if max_val > threshold:
                z = np.argmax(A_scan)  # z position of max amplitude
                surface_coords.append((z, y, x))  # save full coordinate

    if not surface_coords:
        raise ValueError("No surface points found above the threshold.")

    # Convert to NumPy array
    surface_coords_np = np.array(surface_coords)

    # Extract coordinates
    Z = surface_coords_np[:, 0]
    Y = surface_coords_np[:, 1]
    X = surface_coords_np[:, 2]

    # Build matrix A and vector Z for least squares fit
    A = np.c_[X, Y, np.ones_like(X)]
    C, _, _, _ = np.linalg.lstsq(A, Z, rcond=None)  # C = [a, b, c]

    a, b, c = C
    print(f"Fitted plane: z = {a:.4f}x + {b:.4f}y + {c:.4f}")

    if return_points:
        return a, b, c, surface_coords_np
    else:
        return a, b, c
    
def plane_orientation(a, b):
    #indica al documentar que es a y que es b
    angle_yz = np.degrees(np.arctan(b))  # inclination wrt YZ plane
    angle_xz = np.degrees(np.arctan(a))  # inclination wrt XZ plane
    
    print(f"Inclination wrt YZ plane: {angle_yz:.2f} degrees")
    print(f"Inclination wrt XZ plane: {angle_xz:.2f} degrees")
    
    return angle_yz, angle_xz

In [8]:
puntos_maximos = UT_plane(volume_UT, threshold=60)
Orientation_yz, Orientation_xz=plane_orientation(puntos_maximos[0], puntos_maximos[1])

Fitted plane: z = -0.0414x + 0.0721y + 80.2237
Inclination wrt YZ plane: 4.13 degrees
Inclination wrt XZ plane: -2.37 degrees
